In [3]:
import numpy as np

## Iterating over arrays
Done using `nditer`

### Single array iteration

In [4]:
a = np.arange(6).reshape(2,3)
for x in np.nditer(a):
    print(x, end=' ')

0 1 2 3 4 5 

In [5]:
# We can change order of visiting elements
# K is a standard numpy
# F is for Fortran order
# C is for C language order - memory layout
print('K order:')
for x in np.nditer(a, order='K'):
    print(x, end=' ')
print('\nF order:')
for x in np.nditer(a, order='F'):
    print(x, end=' ')
print('\nC order:')
for x in np.nditer(a, order='C'):
    print(x, end=' ')

K order:
0 1 2 3 4 5 
F order:
0 3 1 4 2 5 
C order:
0 1 2 3 4 5 

### Modifying array values

By default nditer is in read-only mode

In [6]:
a = np.arange(6).reshape(2,3)
print(a)
with np.nditer(a, op_flags=["readwrite"]) as it:
   for x in it:
       x[...] = x + 1
print(a)

[[0 1 2]
 [3 4 5]]
[[1 2 3]
 [4 5 6]]


In [7]:
a = np.zeros((100,100), dtype='int16')
with np.nditer(a, op_flags=["readwrite"]) as it:
    i = 0
    for x in it:
        x[...] = i
        i +=1
print(a)

[[   0    1    2 ...   97   98   99]
 [ 100  101  102 ...  197  198  199]
 [ 200  201  202 ...  297  298  299]
 ...
 [9700 9701 9702 ... 9797 9798 9799]
 [9800 9801 9802 ... 9897 9898 9899]
 [9900 9901 9902 ... 9997 9998 9999]]


So I can use `nditer` without `with` but only if I need read-only access. If I want to write I should to it line in example above ^

This should also work for 3D arrays

In [8]:
a_3d = np.zeros((5,5,5), dtype='int8')
with np.nditer(a_3d, op_flags=["readwrite"], order='F') as it:
    i = 0
    for x in it:
        x[...] = i
        i +=1
print(a_3d)

[[[  0  25  50  75 100]
  [  5  30  55  80 105]
  [ 10  35  60  85 110]
  [ 15  40  65  90 115]
  [ 20  45  70  95 120]]

 [[  1  26  51  76 101]
  [  6  31  56  81 106]
  [ 11  36  61  86 111]
  [ 16  41  66  91 116]
  [ 21  46  71  96 121]]

 [[  2  27  52  77 102]
  [  7  32  57  82 107]
  [ 12  37  62  87 112]
  [ 17  42  67  92 117]
  [ 22  47  72  97 122]]

 [[  3  28  53  78 103]
  [  8  33  58  83 108]
  [ 13  38  63  88 113]
  [ 18  43  68  93 118]
  [ 23  48  73  98 123]]

 [[  4  29  54  79 104]
  [  9  34  59  84 109]
  [ 14  39  64  89 114]
  [ 19  44  69  94 119]
  [ 24  49  74  99 124]]]


## Tracking an index or multi-index

`nditer` can start tracking index IF it's given `f_index` or `multi_index` flag

In [9]:
a = np.arange(27).reshape(3,3,3)
it = np.nditer(a, flags=['multi_index']) #nd index
for x in it:
    print(f"{x} {it.multi_index}", end=" > ")

0 (0, 0, 0) > 1 (0, 0, 1) > 2 (0, 0, 2) > 3 (0, 1, 0) > 4 (0, 1, 1) > 5 (0, 1, 2) > 6 (0, 2, 0) > 7 (0, 2, 1) > 8 (0, 2, 2) > 9 (1, 0, 0) > 10 (1, 0, 1) > 11 (1, 0, 2) > 12 (1, 1, 0) > 13 (1, 1, 1) > 14 (1, 1, 2) > 15 (1, 2, 0) > 16 (1, 2, 1) > 17 (1, 2, 2) > 18 (2, 0, 0) > 19 (2, 0, 1) > 20 (2, 0, 2) > 21 (2, 1, 0) > 22 (2, 1, 1) > 23 (2, 1, 2) > 24 (2, 2, 0) > 25 (2, 2, 1) > 26 (2, 2, 2) > 

In [10]:
a = np.arange(6).reshape(2,3)
it = np.nditer(a, flags=['f_index']) # 1d index?
for x in it:
    print("%d <%d>" % (x, it.index), end=' \\ ')

0 <0> \ 1 <2> \ 2 <4> \ 3 <1> \ 4 <3> \ 5 <5> \ 

In [11]:
a = np.zeros((3,3), dtype='int8')
with np.nditer(a, flags=['multi_index'], op_flags=['writeonly']) as it:
    for x in it:
        # print(f'multi_index[0] {it.multi_index[0]} and multi_index[1] is {it.multi_index[1]}', end = ', ')
        x[...] = it.multi_index[1] - it.multi_index[0]
a

array([[ 0,  1,  2],
       [-1,  0,  1],
       [-2, -1,  0]], dtype=int8)

`multi_index` and `index` (i assume) are both just list, that you can access values of

__NOTE__: Tracking an index or multi-index is incompatible with using an external loop, because it requires a different index value per element. If you try to combine these flags, the nditer object will raise an exception.

# Using an external loop

Speaking of external loops! Using flag `external_loop` numpy can let us handle with chunks of memory at once. So instead of giving us one item at times it can give us whole chunk of items. Size of chunks will depend on how data is store and it what order we want to access it.

In [12]:
a = np.arange(6).reshape(2,3)
for x in np.nditer(a, flags=['external_loop']):
    print(x, end=' ')

[0 1 2 3 4 5] 

In case above, we got a chunk of data that is equal to a whole list.

In [13]:
a = np.arange(6).reshape(2,3)
print(a)
for x in np.nditer(a, flags=['external_loop'], order='F'):
    print(x, end=' ')

[[0 1 2]
 [3 4 5]]
[0 3] [1 4] [2 5] 

But if we change the order of iterating to F (column-major) we get chunks of that are way smaller, because we need to travel on our list not in c order.

- **TODO** maybe I need to comeback here? Explore WHY would I need to use external_loop cuz to me it seems that everything I would like to do is already implemented as a numpy function (e.i. adding two arrays together, multiplaying whole array by `x`)